In [13]:
from sklearn.ensemble import RandomForestClassifier
import os
os.chdir('/Users/renalkakhan/Documents/GitHub/CS598_DLH_Project/')
from dataset.preprocessing.data_preprocessing import DataPreprocessing
from dataset.preprocessing.tf_idf_all_feature_matrix_gen import TFIDFFeatureGeneration
from dataset.preprocessing.word2vec_embeddings_gen import Word2VecFeatureGeneration
from sklearn.metrics import f1_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import csv
import pandas as pd

In [14]:
class RandomForest:
    def __init__(self, x_train, y_train, x_test, y_test):
        self.rf = RandomForestClassifier(n_estimators=100, random_state=42)
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        print(self.x_train.shape, self.y_train.shape, self.x_test.shape, self.y_test.shape)
        
    def train(self):
        self.rf.fit(self.x_train, self.y_train)

    def test_and_evaluate(self):
        y_pred = self.rf.predict(self.x_test)
        f1_macro = f1_score(self.y_test, y_pred, average='macro')
        f1_micro = f1_score(self.y_test, y_pred, average='micro')
        #print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")
        return f1_macro, f1_micro

In [15]:
dataPreprocessing = DataPreprocessing()
dataPreprocessing.preprocess_data('./dataset/train/train_data_intuitive.csv', 'train_preprocessed.csv')
dataPreprocessing.preprocess_data('./dataset/test/test_data_intuitive.csv', 'test_preprocessed.csv')

In [16]:
train_preprocessed_df = pd.read_csv('preprocessed_train.csv')
test_preprocessed_df  = pd.read_csv('preprocessed_test.csv')

In [17]:
morbidities = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'Obesity', 'OSA', 'PVD', 'Venous-Insufficiency']

In [18]:
column_headings = ["Morbidity Class", "RF_Macro F1", "RF_Micro F1"]

with open("performance_RF.csv", "w", newline="") as file:
    writer = csv.writer(file)

    # write the RF heading and the subheadings for Micro F1 and Macro F1
    writer.writerow([column_headings[0], column_headings[1], column_headings[2]])
    
for morbidity in morbidities:
    X, Y, words = TFIDFFeatureGeneration(train_preprocessed_df, morbidity).tf_idf_matrix_gen()

    # add KFold cross validation
    skf = KFold(n_splits=10, shuffle=True, random_state=42)

    f1_macro_list = []
    f1_micro_list = []
    for train_idx, val_idx in skf.split(X, Y):
        X_train_fold, Y_train_fold = X[train_idx], Y[train_idx]
        X_val_fold, Y_val_fold = X[val_idx], Y[val_idx]

        # Training RF using TF-IDF Representation
        rf_obj = RandomForest(X_train_fold, Y_train_fold, X_val_fold, Y_val_fold)
        rf_obj.train()

        f1_macro, f1_micro = rf_obj.test_and_evaluate()

        f1_macro_list.append(f1_macro)
        f1_micro_list.append(f1_micro)

    f1_macro = np.mean(f1_macro_list)
    f1_micro = np.mean(f1_micro_list)
    print(f"Macro F1 score: {f1_macro} and Micro F1 Score {f1_micro}")

    row_heading = morbidity

    # data to be written to the CSV file
    data = [f1_macro, f1_micro]

    with open("performance_RF.csv", "a", newline="") as file:
        writer = csv.writer(file)
        row = [row_heading]
        row.extend(data)
        writer.writerow(row)


    

(611, 600) (611,) Counter({0.0: 502, 1.0: 70, -1.0: 39})
(549, 600) (549,) (62, 600) (62,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
Macro F1 score: 0.4719866929194271 and Micro F1 Score 0.8592543627710206
(611, 600) (611,) Counter({1.0: 325, 0.0: 223, -1.0: 63})
(549, 600) (549,) (62, 600) (62,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
(550, 600) (550,) (61, 600) (61,)
Macro F1 score: 0.5660836695083236 and Micro F1 Score 0.8134584875727129
(611, 600) (611,) Counter({-1.0: 368, 1.0: 243})
(549, 600)